# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121742e+02     1.553517e+00
 * time: 0.0934760570526123
     1     1.106535e+01     8.650723e-01
 * time: 0.990792989730835
     2    -1.275862e+01     1.009868e+00
 * time: 1.1428840160369873
     3    -3.458134e+01     7.774515e-01
 * time: 1.3157479763031006
     4    -4.857119e+01     5.116347e-01
 * time: 1.4733550548553467
     5    -5.736638e+01     2.046619e-01
 * time: 1.626626968383789
     6    -6.000338e+01     1.047127e-01
 * time: 1.739583969116211
     7    -6.095896e+01     6.184132e-02
 * time: 1.8596999645233154
     8    -6.135452e+01     7.363773e-02
 * time: 1.9737310409545898
     9    -6.165758e+01     3.844136e-02
 * time: 2.0960090160369873
    10    -6.185244e+01     2.803859e-02
 * time: 2.2165091037750244
    11    -6.199065e+01     2.018065e-02
 * time: 2.3296549320220947
    12    -6.205283e+01     2.010708e-02
 * time: 2.4517979621887207
    13    -6.211000e+01     1.393567e-02
 * time: 2.5715830326080

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557685
    AtomicNonlocal      14.8522644
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485378 
    Xc                  -19.3336823

    total               -62.261666462754
